In [37]:
import csv
import io
import os
import regex as re

from collections import namedtuple
from functools import partial
from os.path import expanduser, isfile, join, realpath
from pprint import pprint
from sqlite3 import Connection

import pyperclip

from bs4 import BeautifulSoup

DOWNLOADS_DIR = realpath(expanduser("~/Downloads"))
TEMP_DIR = expanduser("~/temp")

In [39]:
def resolve_filename(filename):
    if filename.startswith("~"):
        return expanduser(filename)
    else:
        return filename
    
def with_resolution(function):
    def resolution_function(filename, *args, **kwargs):
        resolved_name = resolve_filename(filename)
        return function(resolved_name, *args, **kwargs)
    return resolution_function

def with_real_path(function):
    def wrapped_function(filename, *args, **kwargs):
        real_path = realpath(filename)
        return function(real_path, *args, **kwargs)
    return wrapped_function

# def resolve_link(filename):
#     if os.path.islink(filename):
#         return os.readlink(filename)
#     else:
#         return filename

@with_resolution
def read_all_lines(filename, delete_trailing_carriage_returns=True):
    with open(filename, mode="rt") as fp:
        if delete_trailing_carriage_returns:
            return [line[:-1] for line in  fp.readlines()]
        else:
            return fp.readlines()
        
def remove_empty_and_whitespace_lines(lines):
    return [line for line in lines if len(line.strip()) > 0]

@with_resolution
def write_csv(filename, fieldnames, rows):
    with open(filename, mode="w", newline="") as fp:
        writer = csv.DictWriter(fp, fieldnames)
        writer.writeheader()
        writer.writerows(rows)
        
@with_resolution
def change_ext(filename, new_ext):
    filename_wo_ext, _ = os.path.splitext(filename)
    if new_ext[0] == ".":
        return f"{filename_wo_ext}{new_ext}"
    else:
        return f"{filename_wo_ext}.{new_ext}"

def list_files(directory=".", filter=lambda f:f):
    for f in sorted(os.listdir(directory), key=lambda s : s.lower()):
        if isfile(f) and filter(f):
            print(f)
            
@with_resolution
def read_text_file(filename):
    with open(filename) as fp:
        return fp.read()
            
@with_resolution
def read_html_file(filename):
    with open(filename) as fp:
        return BeautifulSoup(filename, "html.parser")
    
def read_html_text(html_text):
        return BeautifulSoup(html_text, "html.parser")
    
def list_to_string(input_list):
    bytes_written = 0
    with io.StringIO() as sp:
        for line in input_list:
            bytes_written += sp.write(f"{line}\n")
            
        return (bytes_written, sp.getvalue())

In [ ]:
write_csv(change_ext(TO_BE_PARSED, "csv"), ParsedFile._fields, [row._asdict() for row in rows])

In [87]:
def is_icloud_file(filename):
    return filename.startswith(".") and filename.endswith(".icloud")

def get_name_for_icloud_file(filename):
    return filename[1:-7]

def to_lower(s):
    return s.lower()

def sort_ci(string_list):
    return sorted(string_list, sort=to_lower)

@with_real_path
def get_files(folder):
     return [f for f in os.listdir(folder) if isfile(join(folder, f))]
    
def format_long_line(long_title, maxlength=80, indent=4):
    leader = indent * " "
    long_title_length = len(long_title)
    if long_title_length < maxlength:
        return long_title
    
    long_title_words = ((w, len(w)) for w in long_title.split())
    
    line, lines = "", []
    for word, word_length in long_title_words:
        if len(line) + word_length > maxlength:
            lines.append(line)
            line = leader
            
        line += word + " "
    
    # append the last line after exiting the loop
    lines.append(line)
    return "\n".join([line.rstrip() for line in lines])

def print_long_lines(long_lines):
    for line in long_lines:
        print(format_long_line(line))

files = get_files(DOWNLOADS_DIR)
# files = [f for f in os.listdir(os.readlink(os.path.expanduser("~/Downloads"))) if os.path.isfile(f)]
icloud_files = sorted([get_name_for_icloud_file(f) for f in files if is_icloud_file(f)])
#icloud_files = [get_name_for_icloud_file(f) for f in files if f.startswith(".") and f.endswith(".icloud")]
local_files = sorted([f for f in files if not is_icloud_file(f)])

#pprint(icloud_files)
# pprint(local_files)
#pprint([f for f in icloud_files if os.path.splitext(f)[1].lower() in [".pdf", ".mobi", ".epub"]])
zlib_files = [join(DOWNLOADS_DIR, f) for f in local_files if os.path.splitext(f)[1].lower() 
                                     in [".pdf", ".mobi", ".epub"]
                                     and re.search(r"\(z-lib.org\)", f)]

zlib_files.sort(key=to_lower)

delete_zlib = partial(re.sub, r"\s*\(z-lib.org\)\s*", "")
rename_spec = [(file, delete_zlib(file)) for file in zlib_files]

pprint(zlib_files, width=120)
# print(len(zlib_files))

# for rs in rename_spec:
#     os.rename(*rs)



[]


In [38]:
HDMI_MODE_FIELDS = "hdmi_mode resolution frequency screen_aspect notes".split()

HDMI_mode = namedtuple("HDMI_mode", HDMI_MODE_FIELDS)

hdmi_modes_html_text = read_text_file("~/temp/hdmi_modes.html")
soup = read_html_text(hdmi_modes_html_text)
rows = soup.body.table.find_all("tr")
hdmi_modes_rows = [HDMI_mode(*[td.string for td in row.find_all("td")])
                                         for row in rows[1:]]

write_csv("~/temp/hdmi_modes.csv", HDMI_MODE_FIELDS, [row._asdict() for row in hdmi_modes_rows])


In [100]:
CHAPTER_NUMBER_RE = re.compile(r"\((\d+)\)")
LAST_NAME_RE = re.compile(r"^\.\s+(\w+)\s*$")
OUTSTANDING_HEADING = "== outstanding"

def get_chapter_number(line):
    match = CHAPTER_NUMBER_RE.search(line)
    if match:
        return int(match[1])
    else:
        return None
    
def get_last_name(full_name):
    return LAST_NAME_RE.search(full_name)[0]
    
def get_chapter_author_titles(chapter_numbers):
    with Connection(TOC_FILE) as cn:
        for chapter in chapter_numbers:
            cur = cn.execute(GET_CHAPTER_TITLES_SQL, (chapter,))
            yield cur.fetchone()


GROUPINGS_FILE = "~/Repos/music-and-letters/groupings.adoc"
TOC_FILE = expanduser("~/temp/strauss_toc.db")

GET_CHAPTER_TITLES_SQL = "SELECT author, chapter, title" \
                         "FROM v_chapters " \
                         "WHERE chapter = ?;"

ChapterAuthorTitle = namedtuple("ChapterAuthorTitle", "chapter author title")

def ChapterAuthorTitle__repr__(self):
    return f"{get_last_name(self.author)} _({self.title})_ ({self.chapter})"

ChapterAuthorTitle.__repr__ = ChapterAuthorTitle__repr__

lines = {line.strip() : line_num for line_num, line 
                          in enumerate(read_all_lines(GROUPINGS_FILE), start=1)
                          if len(line.strip()) and get_chapter_number(line)
                          or line.startswith("== outstanding")}

# print("lines")
# pprint(lines)

outstanding_line_num = lines[OUTSTANDING_HEADING]

present = [key for key in lines.keys() 
            if lines[key] != OUTSTANDING_HEADING
            and lines[key] < outstanding_line_num]
absent = [key for key in lines.keys() 
           if lines[key] != OUTSTANDING_HEADING
           and lines[key] > outstanding_line_num]

# pprint(present)
pyperclip.copy("\n".join(absent))

# chapters = set(sorted([get_chapter_number(line) for line in lines]))
# print(chapters)
# missing = [c for c in range(1, 37) if not c in chapters]
# print(missing)

# results = list(get_chapter_author_titles(missing))
# pprint(results)

# chapter_author_titles = [ChapterAuthorTitle(*c) for  in results]

# bytes_written, output = list_to_string(chapter_author_titles)
# print(f"{bytes_written} bytes written")
# print(output)
# pyperclip.copy()